# Yahtzee testing - Compute probabilities
The purpose of this part is to facilitate the computation of probabilities of rolls of Yahtzee, given any current state of the dice, including any frozen dice.

## 1. Imports needed

In [1]:
import numpy as np
import pandas as pd

## 2. The table

This will go as follows:

1. Need to populate a column with every 5-digit combination of integers between 1 and 6.
2. Each row will be assigned probability of ${6^{-5}}$
3. A string representation of a dice roll will be passed, and joined to all of the rows which correspond to the dice roll. 
4. The probabilities of the rows will be summed up to the joined level, with an adjustment of $6^{k}$ applied if any $k$ dice are frozen.